In [1]:
import os
import datetime

from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams

from easysteer.steer import StatisticalControlVector
from transformers import AutoTokenizer
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-12 21:42:15 [__init__.py:244] Automatically detected platform cuda.


In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
)

In [3]:
model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
# vector_path="/home/meixinyu/EasySteer/vectors/thinking_switch_pca_MATH-500.gguf" #change to your path
vector_path="MATH500.gguf" #change to your path

control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([-0.01747144, -0.06540639,  0.03629721, ..., -0.01541545,
        -0.02380189, -0.01436467], shape=(1536,), dtype=float32), 1: memmap([ 0.00541517, -0.00878717,  0.00869997, ..., -0.02287901,
        -0.02009154, -0.0380066 ], shape=(1536,), dtype=float32), 2: memmap([ 0.0027614 , -0.02800751,  0.02129964, ...,  0.06221821,
        -0.00991458, -0.04693417], shape=(1536,), dtype=float32), 3: memmap([-0.0152957 , -0.00920365,  0.05090223, ...,  0.01643756,
        -0.01623485, -0.03401608], shape=(1536,), dtype=float32), 4: memmap([-0.00275314,  0.00519124,  0.0310943 , ...,  0.0205216 ,
        -0.02483266, -0.01082839], shape=(1536,), dtype=float32), 5: memmap([-0.00594618,  0.0570037 ,  0.0120637 , ..., -0.00586054,
        -0.03283538, -0.02633283], shape=(1536,), dtype=float32), 6: memmap([-0.00142239, -0.01229366, -0.04342571, ...,  0.01767888,
        -0.03308094, -0.02300258], shape=(1536,), dtype

In [4]:
steer_vector_request_pos = SteerVectorRequest(
            steer_vector_name="thinking_switch_pos",
            steer_vector_id=1,
            steer_vector_local_path=vector_path,
            scale=1.0,
            prefill_trigger_tokens="-1",
            prefill_trigger_positions=[-1],
            generate_trigger_tokens="-1",
            debug=False,
            algorithm='direct'
        )
steer_vector_request_neg = SteerVectorRequest(
            steer_vector_name="thinking_switch_neg",
            steer_vector_id=2,
            steer_vector_local_path=vector_path,
            scale=-1.0,
            prefill_trigger_tokens="-1",
            prefill_trigger_positions=[-1],
            generate_trigger_tokens="-1",
            debug=False,
            algorithm='direct'
        )

In [5]:
sampling_params = SamplingParams(temperature=0.0,max_tokens=4096)
prompt_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
prompt = "Find the greatest common divisor of $3339$, $2961$, and $1491$."
# prompt = "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."
input = prompt_template % prompt

In [6]:
llm = LLM(model=model_path, enable_steer_vector=True, tensor_parallel_size=1)
output_base = llm.generate(
    input,
    sampling_params
)
generated_text_base = output_base[0].outputs[0].text

output_pos = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_pos
            )
generated_text_pos = output_pos[0].outputs[0].text

output_neg = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_neg
            )
generated_text_neg = output_neg[0].outputs[0].text

INFO 07-12 21:42:25 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 07-12 21:42:25 [config.py:1472] Using max model len 32768
WARNING 07-12 21:42:25 [arg_utils.py:1770] --enable-steer-vector is not supported by the V1 Engine. Falling back to V0. 


2025-07-12 21:42:25,378	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-12 21:42:25 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_model_name=/data/zju-46/she

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.13it/s]


INFO 07-12 21:42:28 [default_loader.py:272] Loading weights took 1.03 seconds


INFO 07-12 21:42:28 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.184133 seconds
INFO 07-12 21:42:30 [worker.py:295] Memory profiling takes 1.43 seconds
INFO 07-12 21:42:30 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-12 21:42:30 [worker.py:295] model weights take 2.89GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.05GiB; the rest of the memory reserved for KV Cache is 37.68GiB.
INFO 07-12 21:42:30 [executor_base.py:115] # cuda blocks: 88185, # CPU blocks: 9362
INFO 07-12 21:42:30 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.06x
INFO 07-12 21:42:34 [model_runner.py:1592] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider dec

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.85it/s]

INFO 07-12 21:42:53 [model_runner.py:1755] Graph capturing finished in 19 secs, took 0.24 GiB
INFO 07-12 21:42:53 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 24.11 seconds



Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.44s/it, est. speed input: 6.37 toks/s, output: 85.60 toks/s]


In [7]:
print(generated_text_base)

To find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\), we use the Binomial Theorem. The Binomial Theorem states that:

\[
(a + b)^n = \sum_{k=0}^{n} \binom{n}{k} a^{n-k} b^k
\]

In our case, \(a = 10x^3\), \(b = -\frac{1}{2x^2}\), and \(n = 5\). We need to find the term in the expansion where the power of \(x\) is zero (the constant term).

The general term in the expansion is given by:

\[
\binom{5}{k} (10x^3)^{5-k} \left(-\frac{1}{2x^2}\right)^k
\]

Simplifying the general term, we get:

\[
\binom{5}{k} 10^{5-k} x^{3(5-k)} \left(-\frac{1}{2}\right)^k x^{-2k} = \binom{5}{k} 10^{5-k} \left(-\frac{1}{2}\right)^k x^{15-3k-2k} = \binom{5}{k} 10^{5-k} \left(-\frac{1}{2}\right)^k x^{15-5k}
\]

We need the exponent of \(x\) to be zero for the term to be constant. Therefore, we set the exponent equal to zero:

\[
15 - 5k = 0
\]

Solving for \(k\), we get:

\[
15 = 5k \implies k = 3
\]

Now, we substitute \(k = 3\) back into the general term to find the consta

In [8]:
print(generated_text_pos)

To find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\), we use the Binomial Theorem. The Binomial Theorem states that:

\[
(a + b)^n = \sum_{k=0}^{n} \binom{n}{k} a^{n-k} b^k
\]

In our case, \(a = 10x^3\), \(b = -\frac{1}{2x^2}\), and \(n = 5\). We need to find the term in the expansion where the power of \(x\) is zero (the constant term).

The general term in the expansion is given by:

\[
T_k = \binom{5}{k} (10x^3)^{5-k} \left(-\frac{1}{2x^2}\right)^k
\]

Simplifying the general term, we get:

\[
T_k = \binom{5}{k} (10)^{5-k} (x^3)^{5-k} \left(-\frac{1}{2}\right)^k (x^{-2})^k
\]

\[
T_k = \binom{5}{k} (10)^{5-k} (-1)^k \left(\frac{1}{2}\right)^k x^{3(5-k) - 2k}
\]

\[
T_k = \binom{5}{k} (10)^{5-k} (-1)^k \left(\frac{1}{2}\right)^k x^{15 - 5k - 2k}
\]

\[
T_k = \binom{5}{k} (10)^{5-k} (-1)^k \left(\frac{1}{2}\right)^k x^{15 - 7k}
\]

For the term to be a constant, the exponent of \(x\) must be zero:

\[
15 - 7k = 0
\]

Solving for \(k\):

\[
15 = 7k
\

In [9]:
print(generated_text_neg)

To find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\), we use the Binomial Theorem. The Binomial Theorem states that:

\[
(a + b)^n = \sum_{k=0}^{n} \binom{n}{k} a^{n-k} b^k
\]

In our case, \(a = 10x^3\), \(b = -\frac{1}{2x^2}\), and \(n = 5\). We need to find the term in the expansion where the power of \(x\) is zero (the constant term).

The general term in the expansion is given by:

\[
\binom{5}{k} (10x^3)^{5-k} \left(-\frac{1}{2x^2}\right)^k
\]

Simplifying the general term, we get:

\[
\binom{5}{k} 10^{5-k} x^{3(5-k)} \left(-\frac{1}{2}\right)^k x^{-2k} = \binom{5}{k} 10^{5-k} \left(-\frac{1}{2}\right)^k x^{15-3k-2k} = \binom{5}{k} 10^{5-k} \left(-\frac{1}{2}\right)^k x^{15-5k}
\]

We need the exponent of \(x\) to be zero for the term to be constant. Therefore, we set the exponent equal to zero:

\[
15 - 5k = 0
\]

Solving for \(k\), we get:

\[
15 = 5k \implies k = 3
\]

Now, we substitute \(k = 3\) back into the general term to find the consta

In [10]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename=f'fractreason_GSM8K_{timestamp}.txt'
# filename=f'../../temp/fractreason_GSM8K_{timestamp}.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write("=== Base ===\n")
    f.write(generated_text_base + "\n\n")
    print(len(generated_text_base))

    f.write("=== Positive: Slow thinking ===\n")
    f.write(generated_text_pos + "\n\n")
    print(len(generated_text_pos))

    f.write("=== Negative: Fast thinking ===\n")
    f.write(generated_text_neg + "\n")
    print(len(generated_text_neg))


1272
1275
1272
